# Amber Prep Server - Boltz-2 MD Workflow

This notebook demonstrates the complete MD workflow for Boltz-2 predicted structures:

1. **Load** - Read Boltz-2 mmCIF output file
2. **Prepare** - Clean protein, parameterize ligands (PDBFixer + GAFF2)
3. **Merge** - Combine protein and ligand structures
4. **Solvate** - Add water box and ions (packmol-memgen)
5. **Build** - Generate Amber topology (tleap)
6. **Simulate** - Run MD with OpenMM (NPT ensemble)
7. **Visualize** - Trajectory animation

## Test Case: Boltz-2 Prediction

- **Input**: `boltz_results_ligand/predictions/ligand/ligand_model_0.cif`
- **Format**: mmCIF (Boltz-2 output)
- **Goal**: Process predicted protein-ligand complex → solvate → run short MD

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

from pathlib import Path
import json
import asyncio

# Boltz-2 output file
BOLTZ2_CIF = Path("boltz_results_ligand/predictions/ligand/ligand_model_0.cif")

print(f"Test configuration:")
print(f"  Input file: {BOLTZ2_CIF}")
print(f"  Exists: {BOLTZ2_CIF.exists()}")
print(f"  Note: Processing Boltz-2 mmCIF output")

In [ ]:
# Verify dependencies
print("Checking dependencies...")

required = ["gemmi", "rdkit", "dimorphite_dl", "pdbfixer"]
for pkg in required:
    try:
        __import__(pkg.replace("-", "_"))
        print(f"  ✓ {pkg}")
    except ImportError:
        print(f"  ✗ {pkg}")

# Check AmberTools
print("\nAmberTools:")
from common.base import BaseToolWrapper
for tool in ["antechamber", "parmchk2", "pdb4amber", "tleap", "packmol-memgen"]:
    wrapper = BaseToolWrapper(tool)
    print(f"  {'✓' if wrapper.is_available() else '✗'} {tool}")

## Step 1: Load Boltz-2 Structure

Read the Boltz-2 mmCIF output file. Unlike PDB files, Boltz-2 outputs use mmCIF format with specific chain naming conventions.

In [ ]:
# Import structure server module
import importlib
import servers.structure_server as structure_module
importlib.reload(structure_module)

print("Loading Boltz-2 structure")
print("=" * 60)
print(f"File: {BOLTZ2_CIF}")
print()

if BOLTZ2_CIF.exists():
    structure_file = str(BOLTZ2_CIF)
    
    # Inspect the structure first
    inspect_molecules = structure_module.inspect_molecules
    inspect_result = inspect_molecules(structure_file)
    
    if inspect_result["success"]:
        summary = inspect_result.get("summary", {})
        print(f"✓ Format: {inspect_result['file_format']}")
        print(f"✓ Total chains: {summary.get('total_chains', len(inspect_result['chains']))}")
        print(f"✓ Proteins: {summary.get('num_protein_chains', 0)}")
        print(f"✓ Ligands: {summary.get('num_ligand_chains', 0)}")
        print(f"✓ Waters: {summary.get('num_water_chains', 0)}")
        print(f"✓ Ions: {summary.get('num_ion_chains', 0)}")
        
        print(f"\n--- Chain Details ---")
        for chain in inspect_result['chains']:
            chain_type = chain.get('chain_type', 'unknown')
            chain_id = chain.get('chain_id', chain.get('name', 'unknown'))
            print(f"  {chain_id}: {chain_type} ({chain.get('num_residues', 0)} res, {chain.get('num_atoms', 0)} atoms)")
    else:
        print(f"✗ Error: {inspect_result['errors']}")
        raise RuntimeError("Failed to inspect structure")
else:
    print(f"✗ File not found: {BOLTZ2_CIF}")
    raise FileNotFoundError(f"Boltz-2 output not found: {BOLTZ2_CIF}")

## Step 2: Prepare Complex

Use `prepare_complex` to:
1. Inspect and split the structure into chains
2. Clean protein chains (PDBFixer + pdb4amber)
3. Prepare ligands (SMILES template matching + pH protonation)
4. Parameterize ligands (antechamber GAFF2 + AM1-BCC)
5. Merge all prepared structures into a single PDB file

**Note**: For Boltz-2 outputs, you may need to provide SMILES manually for ligands not in PDB CCD.

In [ ]:
# Get prepare_complex function
prepare_complex = structure_module.prepare_complex

print("Preparing complex (clean + parameterize)")
print("=" * 60)
print(f"Input: {structure_file}")
print()

# Manual SMILES for ligands not in PDB CCD
# Boltz-2 ligands often need manual SMILES specification
# LIG1 is a tyrosine-like ligand (4-hydroxyphenylalanine)
MANUAL_SMILES = {
    "LIG1": "N[C@@H](Cc1ccc(O)cc1)C(=O)O",  # Tyrosine
}

# Prepare the complex (this may take a few minutes for antechamber)
complex_result = prepare_complex(
    structure_file=structure_file,
    select_chains=None,  # Process all chains
    ph=7.4,
    process_proteins=True,
    process_ligands=True,
    run_parameterization=True,
    ligand_smiles=MANUAL_SMILES if MANUAL_SMILES else None
)

if complex_result["success"]:
    output_dir = Path(complex_result["output_dir"])
    print(f"✓ Job ID: {complex_result['job_id']}")
    print(f"✓ Output: {output_dir}")
    
    # Show protein results
    print(f"\n--- Proteins ({len(complex_result['proteins'])}) ---")
    for p in complex_result["proteins"]:
        status = "✓" if p["success"] else "✗"
        print(f"  {status} Chain {p['chain_id']}: {Path(p['output_file']).name}")
    
    # Show ligand results
    print(f"\n--- Ligands ({len(complex_result['ligands'])}) ---")
    for lig in complex_result["ligands"]:
        status = "✓" if lig["success"] else "✗"
        if lig["success"]:
            print(f"  {status} {lig['ligand_id']}: charge={lig['net_charge']}")
            print(f"      mol2: {Path(lig['mol2_file']).name}")
            print(f"      frcmod: {Path(lig['frcmod_file']).name}")
        else:
            print(f"  {status} {lig.get('ligand_id', 'unknown')}: FAILED")
    
    # Show merge results
    print(f"\n--- Merged Structure ---")
    if complex_result.get("merged_pdb"):
        print(f"  ✓ {Path(complex_result['merged_pdb']).name}")
    else:
        print(f"  ✗ Not available")
else:
    print(f"✗ Error: {complex_result['errors']}")
    raise RuntimeError("Complex preparation failed")

## Step 3: Check Merged Structure

`prepare_complex` automatically merges all prepared structures (protein + ligands) into a single PDB file.
The merged PDB is available as `complex_result["merged_pdb"]`.

In [ ]:
# Import solvation server
import servers.solvation_server as solvation_module
importlib.reload(solvation_module)

solvate_structure = solvation_module.solvate_structure

# Use the merged PDB from prepare_complex
print("Using Merged Structure from prepare_complex")
print("=" * 60)

if complex_result.get("merged_pdb"):
    merged_pdb = complex_result["merged_pdb"]
    print(f"✓ Merged PDB: {Path(merged_pdb).name}")
    
    merge_info = complex_result.get("merge_result", {})
    if merge_info.get("statistics"):
        stats = merge_info["statistics"]
        print(f"✓ Total atoms: {stats.get('total_atoms', 'N/A')}")
        print(f"✓ Total residues: {stats.get('total_residues', 'N/A')}")
        print(f"✓ Total chains: {stats.get('total_chains', 'N/A')}")
else:
    print("✗ No merged PDB found")
    raise RuntimeError("No merged PDB available")

## Step 4: Visualize Merged Complex

Visualize the protein-ligand complex before solvation with py3Dmol.

In [ ]:
# Visualize merged complex with py3Dmol
try:
    import py3Dmol
except ImportError:
    %pip install py3Dmol
    import py3Dmol

print("Visualizing merged complex (before solvation)")
print("=" * 60)

with open(merged_pdb, 'r') as f:
    pdb_content = f.read()

lines = pdb_content.split('\n')
atom_lines = [l for l in lines if l.startswith('ATOM') or l.startswith('HETATM')]
print(f"Total atoms: {len(atom_lines)}")

view = py3Dmol.view(width=800, height=500)
view.addModel(pdb_content, 'pdb')

AMINO_ACIDS = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
               'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS',
               'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']

view.setStyle({'resn': AMINO_ACIDS}, {'cartoon': {'color': 'spectrum'}})

# Get actual residue names from PDB file (3-letter codes used in PDB)
# ligand_id may be longer (e.g., LIG1) but PDB uses 3-char codes (e.g., LIG)
ligand_resnames = set()
for lig in complex_result['ligands']:
    if lig['success']:
        # Use first 3 chars of ligand_id as PDB residue name
        resn = lig['ligand_id'][:3].upper()
        ligand_resnames.add(resn)

# Also extract actual residue names from PDB file
pdb_resnames = set()
for line in atom_lines:
    if len(line) >= 20:
        resn = line[17:20].strip()
        if resn and resn not in AMINO_ACIDS:
            pdb_resnames.add(resn)

# Use PDB file residue names (more reliable)
ligand_resnames = pdb_resnames - {'WAT', 'HOH', 'NA', 'CL', 'Na+', 'Cl-'}
print(f"Ligand residues found in PDB: {ligand_resnames}")

# Style ligands with different colors
lig_colors = {'LIG': 'green', 'SAH': 'cyan'}
for resn in ligand_resnames:
    color = lig_colors.get(resn, 'green')
    view.setStyle({'resn': resn}, {'stick': {'color': color, 'radius': 0.3}})
    view.addResLabels({'resn': resn}, {'fontSize': 12, 'fontColor': 'white', 
                                        'backgroundColor': color, 'backgroundOpacity': 0.8})

view.zoomTo()
view.setProjection('orthographic')

print(f"\n🔹 Protein: Cartoon (spectrum)")
print(f"🔹 Ligands: {sorted(ligand_resnames)} (colored sticks)")

view.show()

## Step 5: Solvate Structure

Add water box (12Å buffer) and ions (0.15M NaCl) using packmol-memgen.

In [ ]:
print("Solvating...")
print("=" * 60)
solvate_result = solvate_structure(
    pdb_file=str(Path(merged_pdb).resolve()),
    output_dir=str(output_dir.resolve()),
    output_name="solvated",
    dist=12.0,
    cubic=True,
    salt=True,
    saltcon=0.15
)

if solvate_result["success"]:
    solvated_pdb = solvate_result["output_file"]
    print(f"✓ Solvated: {Path(solvated_pdb).name}")
    print(f"✓ Atoms: {solvate_result['statistics'].get('total_atoms', 'N/A')}")
else:
    print(f"✗ Solvation failed: {solvate_result['errors']}")
    raise RuntimeError("Solvation failed")

## Step 6: Visualize Solvated System

Visualize the solvated system (protein + ligand + water + ions) with py3Dmol.

In [ ]:
print("Visualizing solvated system")
print("=" * 60)

with open(solvated_pdb, 'r') as f:
    pdb_content = f.read()

lines = pdb_content.split('\n')
atom_lines = [l for l in lines if l.startswith('ATOM') or l.startswith('HETATM')]
print(f"Total atoms: {len(atom_lines)}")

view = py3Dmol.view(width=900, height=600)
view.addModel(pdb_content, 'pdb')

AMINO_ACIDS = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
               'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS',
               'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
WATER = ['WAT', 'HOH']

view.setStyle({'resn': AMINO_ACIDS}, {'cartoon': {'color': 'spectrum'}})
view.setStyle({'resn': WATER}, {'sphere': {'radius': 0.15, 'color': 'lightblue'}})
view.setStyle({'resn': ['NA', 'Na+']}, {'sphere': {'radius': 0.8, 'color': 'purple'}})
view.setStyle({'resn': ['CL', 'Cl-']}, {'sphere': {'radius': 0.8, 'color': 'yellow'}})

# Extract actual ligand residue names from PDB file
pdb_resnames = set()
for line in atom_lines:
    if len(line) >= 20:
        resn = line[17:20].strip()
        if resn and resn not in AMINO_ACIDS:
            pdb_resnames.add(resn)

ligand_resnames = pdb_resnames - {'WAT', 'HOH', 'NA', 'CL', 'Na+', 'Cl-'}

# Style ligands with different colors (same as Step 4)
lig_colors = {'LIG': 'green', 'SAH': 'cyan'}
for resn in ligand_resnames:
    color = lig_colors.get(resn, 'green')
    view.setStyle({'resn': resn}, {'stick': {'color': color, 'radius': 0.3}})
    view.addResLabels({'resn': resn}, {'fontSize': 12, 'fontColor': 'white', 
                                        'backgroundColor': color, 'backgroundOpacity': 0.8})

view.zoomTo()
view.setProjection('orthographic')

print(f"\n🔹 Protein: Cartoon (spectrum)")
print(f"🔹 Ligands: {sorted(ligand_resnames)} (colored sticks)")
print(f"🔹 Water: Spheres (light blue)")
print(f"🔹 Ions: Spheres (Na+=purple, Cl-=yellow)")

view.show()

## Step 7: Build Amber System (tleap)

Generate Amber topology (parm7) and coordinate (rst7) files using tleap.
These files are required for OpenMM simulation with proper force field parameters.

In [ ]:
import servers.amber_server as amber_module
importlib.reload(amber_module)

build_amber_system = amber_module.build_amber_system

ligand_params = []
for lig in complex_result.get("ligands", []):
    if lig.get("success") and lig.get("mol2_file"):
        ligand_params.append({
            "mol2": lig["mol2_file"],
            "frcmod": lig["frcmod_file"],
            "residue_name": lig["ligand_id"][:3].upper()
        })

print(f"Ligand parameters: {len(ligand_params)} ligand(s)")
for i, lp in enumerate(ligand_params):
    print(f"  {i+1}. {lp['residue_name']}: {Path(lp['mol2']).name}")

print("\nBuilding Amber system...")
amber_result = build_amber_system(
    pdb_file=solvate_result["output_file"],
    ligand_params=ligand_params if ligand_params else None,
    box_dimensions=solvate_result.get("box_dimensions"),
    water_model="tip3p",
    output_name="system"
)

print(f"\nSuccess: {amber_result['success']}")
if amber_result['success']:
    print(f"Topology: {amber_result.get('parm7')}")
    print(f"Coordinates: {amber_result.get('rst7')}")
else:
    print(f"Errors: {amber_result.get('errors')}")

## Step 8: OpenMM Simulation

Run MD simulation with OpenMM using Amber parm7/rst7 files from Step 7.

**Protocol:**
1. Energy minimization (500 steps)
2. NPT equilibration (5 ps)
3. NPT production (10 ps)

**Parameters:** Temperature 300K, Pressure 1 atm, Timestep 2 fs

In [ ]:
import time

try:
    import openmm as mm
    from openmm import app, unit
    from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, StateDataReporter, DCDReporter, PDBFile
except ImportError:
    %pip install openmm
    import openmm as mm
    from openmm import app, unit
    from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, StateDataReporter, DCDReporter, PDBFile

def select_platform():
    platform_preference = ['CUDA', 'OpenCL', 'CPU']
    print("Checking available platforms...")
    for name in platform_preference:
        try:
            platform = mm.Platform.getPlatformByName(name)
            if name == 'CUDA':
                try:
                    platform.getPropertyDefaultValue('DeviceIndex')
                    print(f"  ✓ {name} available")
                    return platform, name
                except Exception:
                    print(f"  ✗ {name} not available")
                    continue
            elif name == 'OpenCL':
                print(f"  ✓ {name} available")
                return platform, name
            else:
                print(f"  ✓ {name} available")
                return platform, name
        except Exception as e:
            print(f"  ✗ {name} not available")
    raise RuntimeError("No suitable platform found!")

if 'amber_result' in dir() and amber_result.get('success'):
    parm7_file = amber_result['parm7']
    rst7_file = amber_result['rst7']
    
    print("=" * 60)
    print("OpenMM MD Simulation")
    print("=" * 60)
    
    platform, platform_name = select_platform()
    print(f"\n→ Using platform: {platform_name}")
    
    temperature = 300 * unit.kelvin
    pressure = 1 * unit.atmosphere
    timestep = 2 * unit.femtoseconds
    friction = 1 / unit.picosecond
    minimize_max_iter = 500
    equil_steps = 2500
    prod_steps = 1000
    report_interval = 10
    
    print(f"\n[Step 1] Loading system...")
    prmtop = AmberPrmtopFile(parm7_file)
    inpcrd = AmberInpcrdFile(rst7_file)
    print(f"  Atoms: {prmtop.topology.getNumAtoms()}")
    
    print(f"\n[Step 2] Creating system...")
    system = prmtop.createSystem(
        nonbondedMethod=app.PME,
        nonbondedCutoff=10 * unit.angstrom,
        constraints=app.HBonds,
        rigidWater=True
    )
    system.addForce(mm.MonteCarloBarostat(pressure, temperature, 25))
    
    integrator = mm.LangevinMiddleIntegrator(temperature, friction, timestep)
    simulation = Simulation(prmtop.topology, system, integrator, platform)
    simulation.context.setPositions(inpcrd.positions)
    if inpcrd.boxVectors is not None:
        simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
    
    # Save initial structure (from tleap) for trajectory visualization
    initial_pdb = output_dir / "initial_state.pdb"
    state = simulation.context.getState(getPositions=True)
    with open(initial_pdb, 'w') as f:
        PDBFile.writeFile(simulation.topology, state.getPositions(), f)
    print(f"  Initial PDB saved: {initial_pdb.name}")
    
    print(f"\n[Step 3] Energy minimization...")
    simulation.minimizeEnergy(maxIterations=minimize_max_iter)
    simulation.context.setVelocitiesToTemperature(temperature)
    
    dcd_file = output_dir / "trajectory.dcd"
    log_file = output_dir / "simulation.log"
    simulation.reporters.append(DCDReporter(str(dcd_file), report_interval))
    simulation.reporters.append(StateDataReporter(str(log_file), report_interval,
        step=True, time=True, potentialEnergy=True, temperature=True, speed=True))
    simulation.reporters.append(StateDataReporter(sys.stdout, report_interval,
        step=True, time=True, potentialEnergy=True, temperature=True, speed=True, remainingTime=True,
        totalSteps=equil_steps + prod_steps))
    
    print(f"\n[Step 4] Equilibration ({equil_steps * 2 / 1000} ps)...")
    simulation.step(equil_steps)
    
    print(f"\n[Step 5] Production ({prod_steps * 2 / 1000} ps)...")
    simulation.step(prod_steps)
    
    final_pdb = output_dir / "final_state.pdb"
    state = simulation.context.getState(getPositions=True)
    with open(final_pdb, 'w') as f:
        PDBFile.writeFile(simulation.topology, state.getPositions(), f)
    print(f"\n✓ Simulation complete!")
else:
    print("No Amber topology available.")

## Step 9: Trajectory Visualization

Visualize the MD trajectory with py3Dmol animation.
- Protein (cartoon) + ligand (stick) only (water/ions excluded)
- 15 frames sampled from trajectory

In [ ]:
import numpy as np
import tempfile

try:
    import mdtraj as md
except ImportError:
    %pip install mdtraj
    import mdtraj as md

import py3Dmol

if 'output_dir' in dir() and (output_dir / "trajectory.dcd").exists():
    dcd_file = output_dir / "trajectory.dcd"
    
    # Use the same parm7 topology that OpenMM used
    # This ensures perfect consistency between simulation and analysis
    print("Loading trajectory...")
    print(f"  Topology: {Path(parm7_file).name}")
    traj = md.load(str(dcd_file), top=parm7_file)
    print(f"  Frames: {traj.n_frames}")
    print(f"  Atoms: {traj.n_atoms}")
    
    protein_indices = traj.topology.select('protein')
    lig_indices = []
    ligand_resnames = set()
    standard_res = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
                    'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
                    'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL',
                    'WAT', 'HOH', 'NA', 'CL', 'Na+', 'Cl-'}
    for residue in traj.topology.residues:
        if residue.name not in standard_res:
            lig_indices.extend([atom.index for atom in residue.atoms])
            ligand_resnames.add(residue.name)
    lig_indices = np.array(lig_indices) if lig_indices else np.array([], dtype=int)
    
    if len(lig_indices) > 0:
        keep_indices = np.unique(np.concatenate([protein_indices, lig_indices]))
    else:
        keep_indices = protein_indices
    
    traj_subset = traj.atom_slice(keep_indices)
    
    max_frames = 15
    if traj_subset.n_frames > max_frames:
        frame_indices = np.linspace(0, traj_subset.n_frames - 1, max_frames, dtype=int)
        traj_viz = traj_subset[frame_indices]
    else:
        traj_viz = traj_subset
    
    with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
        tmp_path = tmp.name
    
    with open(tmp_path, 'w') as f:
        for frame_idx in range(traj_viz.n_frames):
            frame = traj_viz[frame_idx]
            frame_tmp = tmp_path + f".frame{frame_idx}.pdb"
            frame.save_pdb(frame_tmp, force_overwrite=True)
            with open(frame_tmp, 'r') as ff:
                content = ff.read()
            f.write(f"MODEL     {frame_idx + 1}\n")
            for line in content.split('\n'):
                if not line.startswith('MODEL') and not line.startswith('ENDMDL') and line.strip():
                    f.write(line + '\n')
            f.write("ENDMDL\n")
            Path(frame_tmp).unlink()
    
    with open(tmp_path, 'r') as f:
        pdb_content = f.read()
    Path(tmp_path).unlink()
    
    view = py3Dmol.view(width=800, height=600)
    view.addModelsAsFrames(pdb_content, 'pdb')
    
    aa_list = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
               'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
               'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
    view.setStyle({'resn': aa_list}, {'cartoon': {'color': 'spectrum'}})
    
    lig_colors = ['green', 'cyan', 'magenta', 'orange']
    for i, resn in enumerate(sorted(ligand_resnames)):
        color = lig_colors[i % len(lig_colors)]
        view.setStyle({'resn': resn}, {'stick': {'color': color, 'radius': 0.3}})
        view.addResLabels({'resn': resn}, {'fontSize': 12, 'fontColor': 'white',
                                            'backgroundColor': color, 'backgroundOpacity': 0.8})
    
    view.zoomTo()
    view.setProjection('orthographic')
    view.animate({'loop': 'forward', 'reps': 0, 'interval': 100})
    
    print(f"\n🔹 Protein: Cartoon (spectrum)")
    print(f"🔹 Ligands: {list(ligand_resnames)} (stick)")
    print(f"🔹 Frames: {traj_viz.n_frames} (animated)")
    
    view.show()
else:
    print("Trajectory file not found.")

## Summary

Complete MD workflow from Boltz-2 prediction to trajectory visualization:

| Step | Description | Tool/Method |
|------|-------------|-------------|
| 1 | Load Boltz-2 structure | `inspect_molecules` (gemmi) |
| 2 | Prepare complex | `prepare_complex` (PDBFixer + antechamber) |
| 3 | Check merged | Auto-merged by prepare_complex |
| 4 | Visualize complex | py3Dmol |
| 5 | Solvate | `solvate_structure` (packmol-memgen) |
| 6 | Visualize solvated | py3Dmol |
| 7 | Build Amber system | `build_amber_system` (tleap) |
| 8 | Run MD | OpenMM (NPT) |
| 9 | Visualize trajectory | MDTraj + py3Dmol |

### Notes for Boltz-2 Structures

- **SMILES**: Boltz-2 ligands may not be in PDB CCD. Provide SMILES manually in `MANUAL_SMILES` dict.
- **Chain IDs**: Boltz-2 uses mmCIF format with specific chain naming.
- **Bond Orders**: The workflow uses SMILES template matching to assign correct bond orders.

In [ ]:
print("=" * 60)
print("GENERATED FILES SUMMARY")
print("=" * 60)

print(f"\n📁 Output directory: {output_dir}")

print(f"\n📄 Protein files:")
for p in complex_result["proteins"]:
    if p["success"]:
        print(f"   • {Path(p['output_file']).name}")

print(f"\n📄 Ligand files:")
for lig in complex_result["ligands"]:
    if lig["success"]:
        print(f"   • {lig['ligand_id']}:")
        print(f"     - MOL2: {Path(lig['mol2_file']).name}")
        print(f"     - FRCMOD: {Path(lig['frcmod_file']).name}")

print(f"\n📄 Structure files:")
print(f"   • Merged: {Path(merged_pdb).name}")
print(f"   • Solvated: {Path(solvated_pdb).name}")

print(f"\n📄 Amber files:")
if 'amber_result' in dir() and amber_result.get('success'):
    print(f"   • Topology: {Path(amber_result['parm7']).name}")
    print(f"   • Coordinates: {Path(amber_result['rst7']).name}")

print(f"\n📄 Simulation files:")
if 'dcd_file' in dir():
    print(f"   • Trajectory: {dcd_file.name}")
    print(f"   • Log: {log_file.name}")
    print(f"   • Final state: {final_pdb.name}")

print(f"\n✅ Workflow complete!")